# DecisionTree 공부하기 

## f1.score

In [280]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings('ignore')

In [258]:
import math
from collections import Counter, defaultdict
from functools import partial

In [3]:
card = pd.read_csv('creditcard.csv')

In [4]:
card['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [5]:
card.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [6]:
card.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [7]:
card.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [8]:
card['Time'].value_counts() # 시간

163152.0    36
64947.0     26
68780.0     25
3767.0      21
3770.0      20
            ..
127750.0     1
62260.0      1
62259.0      1
127753.0     1
172792.0     1
Name: Time, Length: 124592, dtype: int64

In [9]:
card['Amount'].value_counts() # 거래량

1.00       13688
1.98        6044
0.89        4872
9.99        4747
15.00       3280
           ...  
62.09          1
552.05         1
1467.75        1
196.68         1
95.63          1
Name: Amount, Length: 32767, dtype: int64

In [10]:
card1 = card.drop_duplicates()

In [11]:
card1

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


--------------------

In [12]:
card['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [150]:
card2 =card.drop_duplicates()

In [151]:
legit = card2[card2['Class'] == 0]
fraud = card2[card2['Class'] == 1]

In [225]:
legit_sample = legit.sample(492*500)

In [226]:
new_dataset = pd.concat([legit_sample, fraud])

In [227]:
new_dataset['Class'].value_counts()

0    246000
1       473
Name: Class, dtype: int64

In [228]:
new_dataset['Class'].value_counts()

0    246000
1       473
Name: Class, dtype: int64

In [229]:
new_dataset2 = new_dataset.drop_duplicates()

In [230]:
new_dataset2.duplicated().value_counts()

False    246473
dtype: int64

In [231]:
new_dataset2.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,246473.000000,246473.000000,246473.000000,246473.000000,246473.000000,246473.000000,246473.000000,246473.000000,246473.000000,246473.000000,...,246473.000000,246473.000000,246473.000000,246473.000000,246473.000000,246473.000000,246473.000000,246473.000000,246473.000000,246473.000000
mean,94867.275840,0.003493,-0.003000,-0.000251,-0.003983,0.002797,-0.002159,-0.001013,-0.000204,-0.003833,...,-0.000343,0.000435,-0.000224,0.000015,-0.000722,-0.000231,0.002015,0.000950,88.177952,0.001919
std,47460.371553,1.955511,1.649555,1.513969,1.417244,1.364159,1.325958,1.213063,1.183181,1.096265,...,0.724201,0.724504,0.627990,0.605794,0.521413,0.481715,0.392892,0.327496,242.199385,0.043765
min,0.000000,-56.407510,-72.715728,-33.680984,-5.683171,-42.147898,-26.160506,-43.557242,-73.216718,-13.434066,...,-34.830382,-9.499423,-44.807735,-2.836627,-10.295397,-2.604551,-22.565679,-11.710896,0.000000,0.000000
25%,54268.000000,-0.916042,-0.600794,-0.889735,-0.851605,-0.689620,-0.769536,-0.552791,-0.208633,-0.647141,...,-0.228235,-0.542751,-0.161843,-0.354603,-0.317875,-0.326953,-0.070650,-0.052694,5.630000,0.000000
50%,84761.000000,0.017349,0.064766,0.178573,-0.024417,-0.052526,-0.276480,0.040924,0.022061,-0.054034,...,-0.028992,0.007586,-0.011192,0.040895,0.014864,-0.052114,0.001649,0.011456,22.000000,0.000000
75%,139333.000000,1.315404,0.803235,1.027598,0.739139,0.612761,0.394284,0.570299,0.325915,0.594569,...,0.186536,0.529358,0.147665,0.440101,0.350328,0.239897,0.091684,0.078712,77.650000,0.000000
max,172792.000000,2.451888,22.057729,9.382558,16.875344,34.801666,23.917837,44.054461,20.007208,15.594995,...,27.202839,10.503090,22.528412,4.022866,7.519589,3.517346,12.152401,33.847808,18910.000000,1.000000


In [232]:
new_dataset['Class'].unique()

array([0, 1], dtype=int64)

---

In [233]:
new_dataset.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [234]:
x = new_dataset.drop(['Class'],axis=1)

In [235]:
y = new_dataset['Class']

In [236]:
x_train, x_test, y_train, y_test =\
train_test_split(x,y,test_size = 0.2, random_state = 0)

In [237]:
model = tree.DecisionTreeClassifier()

In [238]:
y_train

138780    0
256190    0
57592     0
246634    0
6900      0
         ..
201549    0
21194     0
120764    0
180443    0
54651     0
Name: Class, Length: 197178, dtype: int64

In [239]:
model

DecisionTreeClassifier()

In [240]:
model.fit(x_train, y_train)

DecisionTreeClassifier()

In [241]:
model.score(x_train, y_train)

1.0

In [242]:
model.score(x_test, y_test)

0.999046556445887

-

In [264]:
model1 = DecisionTreeClassifier()

In [265]:
model2 = LogisticRegression()

In [266]:
model3 = SVC()

In [268]:
model_list =[('DT',model1),('LR',model2),('SVC',model3)]

In [273]:
vc_model = VotingClassifier(estimators=model_list)

In [281]:
vc_model.fit(x_train, y_train)

VotingClassifier(estimators=[('DT', DecisionTreeClassifier()),
                             ('LR', LogisticRegression()), ('SVC', SVC())])

In [282]:
vc_model.score(x_test, y_test)

0.9991682726442844

-

In [243]:
pred = model.predict(x_test)

In [244]:
from sklearn.metrics import f1_score
#from sklearn.preprocessing import LabelBinarizer # str을 int 로 사용하기 위해 사용?!

In [245]:
f1_score(pred, y_test,average='macro')

0.8792483462131311

--

In [246]:
X = card.drop('Class', axis =1)
Y = card['Class']

In [247]:
real_pred = model.predict(X)

In [248]:
f1_score(Y, real_pred, pos_label = 0)

0.9998698562439104

In [249]:
f1_score(Y, real_pred, pos_label = 1)

0.9268774703557312

In [250]:
f1_score(Y, real_pred,average='macro')

0.9633736632998209

--